<a href="https://colab.research.google.com/github/Mozzer2310/text-mining-cwk/blob/wills-kitchen/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Approach

## Dataset Exploration

We need to install the `datasets` module to download the DialogRE dataset.

In [23]:
!pip install datasets -q

Then we can download the dataset.

In [24]:
from datasets import load_dataset

dataset = load_dataset(
    'dataset-org/dialog_re',
    download_mode='force_redownload',
    trust_remote_code=True,
)

dialog_re.py:   0%|          | 0.00/4.83k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.45k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1073 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/357 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/358 [00:00<?, ? examples/s]

In [25]:
entities = {
    entity
    for split in dataset.values()
    for example in split
    for entity in example['relation_data']['x'] + example['relation_data']['y']
}

## GloVe Embeddings

We first download the GloVe embeddings. TODO: check which pretrained vectors to use.

In [26]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2025-02-20 20:04:03--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-02-20 20:04:03--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  4.88MB/s    in 2m 43s  

2025-02-20 20:06:46 (5.05 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



We can then extract the embeddings.

In [29]:
!python -m spacy init vectors en glove.6B.zip glove_vectors

ℹ Creating blank nlp object for language 'en'
400000it [00:04, 84193.80it/s]
✔ Successfully converted 400000 vectors
✔ Saved nlp object with vectors to output directory. You can now use
the path to it in your config as the 'vectors' setting in [initialize].
/content/glove_vectors


Some example word vectors:

In [33]:
import spacy

glove = spacy.load('glove_vectors')
speaker_1 = glove('Speaker 1')
speaker_2 = glove('Speaker 2')

speaker_1.similarity(speaker_2)

0.9787847605154955